In [3]:
import os
import re
import json
import random
import sys
import pickle
import datetime
import asyncio
import nest_asyncio
import sympy as sp
import gurobipy as gp

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from tqdm.auto import tqdm
from colorama import Fore, Style
from pydantic import BaseModel
from typing import List
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.llms.lmstudio import LMStudio

sys.path.append('../')
from utils import *

nest_asyncio.apply()

In [4]:
DATA_DIR = '../data'
DATASET_NAME = 'LPWP' 
OUTPUT_DIR = '../output'  

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'LPWP.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
# demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in qa_pairs]
answers = [a for _, a in qa_pairs]

2024-09-20 15:07:42.468 | DEBUG    | utils:read_txt_file:15 - Reading file: ../data\LPWP\LPWP.txt
2024-09-20 15:07:42.469 | DEBUG    | utils:read_txt_file:17 - File read successfully: ../data\LPWP\LPWP.txt
2024-09-20 15:07:42.470 | INFO     | utils:get_nl4opt_qas:36 - Number of questions: 288
2024-09-20 15:07:42.471 | INFO     | utils:get_nl4opt_qas:37 - Number of answers: 288


In [5]:
# class Code(BaseModel):
    # reasoning: List[str]
    # code: str     
    
class Result(BaseModel):
    res: float

In [8]:
llm = LMStudio(
    model_name="bartowski/DeepSeek-Coder-V2-Lite-Instruct-GGUF",
    base_url="http://localhost:1234/v1",
    temperature=0.0,
)

In [9]:
prompt_template_str = """You are an expert in optimization problems. Your task is to find the optimal solution to the given problem. If there is no optimial solution, please return inf. Guide the user through the solution step by step.\n"""

prompt_template_str = prompt_template_str + "\nQUESTIOM: {q}"
print(prompt_template_str)

You are an expert in optimization problems. Your task is to find the optimal solution to the given problem. If there is no optimial solution, please return inf. Guide the user through the solution step by step.

QUESTIOM: {q}


In [10]:
program = LLMTextCompletionProgram.from_defaults(
    output_cls=Result,
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=False,
)

In [11]:
results = {}
for i in tqdm(range(len(questions))):
    try:
        results[i] = program(q=questions[i])
    except Exception as e:
        # loguru.logger.error(f"Error for question {i}: {e}")
        results[i] = f"{e}"
        continue

100%|██████████| 288/288 [1:21:37<00:00, 17.00s/it]


In [12]:
results

{0: '1 validation error for Result\n  Invalid JSON: expected value at line 8 column 16 [type=json_invalid, input_value=\'{\\n  "res": {\\n    "obje...es": ["S", "T"]\\n  }\\n}\', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.8/v/json_invalid',
 1: Result(res=5050.0),
 2: '1 validation error for Result\n  Invalid JSON: key must be a string at line 1 column 2 [type=json_invalid, input_value=\'{350 - 6x}{6}\\n     \\\\]\\...\\n```json\\n{"res": 375}\', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.8/v/json_invalid',
 3: '1 validation error for Result\n  Invalid JSON: key must be a string at line 1 column 2 [type=json_invalid, input_value=\'{Discharge} = 0.3x + 0.1...:\\n\\\\[ {\\\\"res\\\\": 3.4}\', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.8/v/json_invalid',
 4: '1 validation error for Result\n  Invalid JSON: key must be a string at line 1 column 2 [type=json_invalid, input_value=

In [13]:
def parse_value_error_message(text: str) -> str:
    # Regular expression to capture everything after "CODE:"
    pattern = r'CODE:\s*(.*)'

    # Extracting the code after "CODE:"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        code = match.group(1).strip()  # Extract the captured group and strip extra spaces
        return code
    else:
        return None

In [145]:
while True:
    error_idx = []
    for k, v in results.items():
        if isinstance(v, str):
            if v.startswith("1 validation error for") or v.startswith("timed out"):
                error_idx.append(k)
    
    print(error_idx)
    if len(error_idx) == 0:
        break

    for ei in tqdm(error_idx):
        try:
            results[ei] = program(q=questions[ei])
        except Exception as e:
            # loguru.logger.error(f"Error for question {i}: {e}")
            results[ei] = f"{e}"
            continue

loguru.logger.info("All questions have been answered")

[0, 2, 3, 4, 7, 8, 9, 10, 11, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 51, 52, 53, 54, 56, 57, 58, 59, 62, 63, 64, 66, 67, 68, 70, 72, 73, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 127, 128, 129, 131, 132, 135, 136, 137, 138, 139, 141, 142, 144, 145, 146, 147, 148, 149, 151, 153, 154, 156, 157, 158, 159, 160, 161, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 198, 199, 200, 201, 202, 204, 205, 206, 208, 209, 210, 211, 212, 216, 217, 219, 220, 221, 225, 226, 227, 228, 230, 231, 233, 234, 235, 236, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 250, 251, 252, 253, 254, 256, 257, 259, 260, 261, 262, 263, 26

  9%|▉         | 22/237 [05:46<54:21, 15.17s/it]  

In [154]:
filename = 'deepseek_coderv2_lpwp_baseline' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(results, f)

In [29]:
# Read from pickel file
filename = 'e2e_codegen_deepseek_coderv2_lpwp_gurobi_2024-09-18-20-33-30.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'rb') as f:
    codes = pickle.load(f)

In [30]:
code_strs = [codes[i].code if isinstance(codes[i], Code) else codes[i] for i in range(len(codes)) ]

In [31]:
prefix = """
import gurobipy as gp
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()
m = gp.Model(env=env)
"""
                
suffix = """
m.optimize()
"""

def complement_code(code: str) -> float:
    return prefix + code + suffix

In [32]:
def clean_code(code: str) -> str:
    temp_code = code
    # Split the code into lines
    pattern = r'\)([a-zA-Z])'
    temp_code = re.sub(pattern, r')\n\1', temp_code)

    cleand_code = []
    for line in temp_code.split('\n'):
        line = line.strip()
        # Replace > < to >= <=
        if line.startswith('m.addConstr') and not re.findall(r'<=|>=', line):
            # print("Not found")
            line = re.sub(r'<', r'<=', line)
            line = re.sub(r'>', r'>=', line)
        # Remove all comments and suffix and prefix terms
        if not line.startswith('```') and not line.startswith('#'):
            cleand_code.append(line)
        else:
            continue
        # Don't support bool expression '=='
        if re.findall(r'==', line):
            cleand_code.remove(line)
        
    cleand_code = '\n'.join(cleand_code)

    # Remove all '{' and '}'
    cleand_code = cleand_code.replace('{', '').replace('}', '')
    return cleand_code

In [33]:
def execute_code(code: str) -> float:
    ex_locals = {}
    exec(code, None, ex_locals)
    
    try:
        return ex_locals["m"].objVal
    except Exception as e:
        # print(e)
        return np.inf

In [34]:
def get_variables(code: List[str]) -> List[str]:
    vars = []
    for line in code:
        if re.findall(r'addVar', line):
            pattern = r'(\w+)\s*='
            matches = re.findall(pattern, line)
            vars.append(matches[0])
    return sp.symbols(vars)

In [35]:
def simplify_code(code: str) -> str:
    simplfied_code = []
    for i, line in enumerate(code.split('\n')):
        if line.startswith('m.addConstr') or line.startswith('m.setObjective'):
            if '/' in line:
                obj_pattern = r'm\.setObjective\(([^,]*)'
                constr_pattern = r'm\.addConstr\((.*)\)'
                if re.findall(obj_pattern, line):
                    matches = re.findall(obj_pattern, line)
                    obj = re.search(r'gp\.GRB\.(\w+)', line).group(1)
                    expr = sp.sympify(matches[0])
                    simplfied_code.append(f"m.setObjective({str(sp.simplify(expr))}, gp.GRB.{obj})")
                if re.findall(constr_pattern, line):
                    matches = re.findall(constr_pattern, line)
                    oper = re.search(r'\s*(>=|<=)\s*', matches[0]).group(1)
                    expr = sp.sympify(matches[0])
                    simplified_expr = str(sp.simplify(expr.lhs - expr.rhs))
                    if match := re.search(r'^\((.*?)\)/', simplified_expr):
                        new_constr = f'{match.group(1)} {oper} {str(0)}'
                        simplfied_code.append('m.addConstr(' + new_constr + ')')
            else:
                simplfied_code.append(line)
        else:
            simplfied_code.append(line)
    return '\n'.join(simplfied_code)

In [36]:
pred_answers = []
for i, code_str in enumerate(code_strs):
    try:
        cl_code = clean_code(code_str)
        si_code = simplify_code(cl_code)
        co_code = complement_code(si_code)
        ans = execute_code(co_code)
        loguru.logger.info(f"question {i} obtain answer")
        pred_answers.append(ans)
    except Exception as e:
        loguru.logger.error(f"Error for question {i}: {e}")
        pred_answers.append("Error")

2024-09-19 15:02:12.702 | ERROR    | __main__:<module>:11 - Error for question 0: name 'budget' is not defined
2024-09-19 15:02:12.704 | INFO     | __main__:<module>:8 - question 1 obtain answer
2024-09-19 15:02:12.707 | INFO     | __main__:<module>:8 - question 2 obtain answer
2024-09-19 15:02:12.709 | INFO     | __main__:<module>:8 - question 3 obtain answer
2024-09-19 15:02:12.711 | INFO     | __main__:<module>:8 - question 4 obtain answer
2024-09-19 15:02:12.713 | INFO     | __main__:<module>:8 - question 5 obtain answer
2024-09-19 15:02:12.714 | INFO     | __main__:<module>:8 - question 6 obtain answer
2024-09-19 15:02:12.716 | INFO     | __main__:<module>:8 - question 7 obtain answer
2024-09-19 15:02:12.717 | INFO     | __main__:<module>:8 - question 8 obtain answer
2024-09-19 15:02:12.719 | INFO     | __main__:<module>:8 - question 9 obtain answer
2024-09-19 15:02:12.721 | INFO     | __main__:<module>:8 - question 10 obtain answer
2024-09-19 15:02:12.724 | INFO     | __main__:<m

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 2 columns and 3 nonzeros
Model fingerprint: 0x5ffc7e62
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+02]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 1000 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+03, best bound 1.000000000000e+03, gap 0.0000%


2024-09-19 15:02:12.809 | INFO     | __main__:<module>:8 - question 51 obtain answer
2024-09-19 15:02:12.823 | ERROR    | __main__:<module>:11 - Error for question 52: name 'y_process2' is not defined
2024-09-19 15:02:12.826 | INFO     | __main__:<module>:8 - question 53 obtain answer
2024-09-19 15:02:12.828 | INFO     | __main__:<module>:8 - question 54 obtain answer


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0xe6cd2430
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)
Found heuristic solution: objective 515.0000000
Found heuristic solution: objective 516.0000000

Root relaxation: objective 5.710000e+02, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

2024-09-19 15:02:12.839 | INFO     | __main__:<module>:8 - question 55 obtain answer
2024-09-19 15:02:12.841 | INFO     | __main__:<module>:8 - question 56 obtain answer


...

2024-09-19 15:02:13.441 | INFO     | __main__:<module>:8 - question 234 obtain answer
2024-09-19 15:02:13.453 | INFO     | __main__:<module>:8 - question 235 obtain answer
2024-09-19 15:02:13.454 | INFO     | __main__:<module>:8 - question 236 obtain answer


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xfcaf9f6b
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 5e+00]
  Objective range  [5e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 5e+02]
Found heuristic solution: objective 750.0000000
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 750 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.500000000000e+02, best bound 7.500000000000e+02, gap 0.0000%


2024-09-19 15:02:13.463 | INFO     | __main__:<module>:8 - question 237 obtain answer
2024-09-19 15:02:13.464 | INFO     | __main__:<module>:8 - question 238 obtain answer
2024-09-19 15:02:13.467 | INFO     | __main__:<module>:8 - question 239 obtain answer
2024-09-19 15:02:13.468 | INFO     | __main__:<module>:8 - question 240 obtain answer
2024-09-19 15:02:13.469 | INFO     | __main__:<module>:8 - question 241 obtain answer
2024-09-19 15:02:13.471 | INFO     | __main__:<module>:8 - question 242 obtain answer
2024-09-19 15:02:13.472 | INFO     | __main__:<module>:8 - question 243 obtain answer
2024-09-19 15:02:13.474 | INFO     | __main__:<module>:8 - question 244 obtain answer
2024-09-19 15:02:13.475 | INFO     | __main__:<module>:8 - question 245 obtain answer
2024-09-19 15:02:13.477 | INFO     | __main__:<module>:8 - question 246 obtain answer
2024-09-19 15:02:13.478 | INFO     | __main__:<module>:8 - question 247 obtain answer
2024-09-19 15:02:13.479 | INFO     | __main__:<module>

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xe20e09e8
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+02]
Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 8 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e+00, best bound 8.000000000000e+00, gap 0.0000%


2024-09-19 15:02:13.507 | INFO     | __main__:<module>:8 - question 263 obtain answer
2024-09-19 15:02:13.509 | INFO     | __main__:<module>:8 - question 264 obtain answer
2024-09-19 15:02:13.510 | INFO     | __main__:<module>:8 - question 265 obtain answer
2024-09-19 15:02:13.511 | INFO     | __main__:<module>:8 - question 266 obtain answer
2024-09-19 15:02:13.513 | INFO     | __main__:<module>:8 - question 267 obtain answer
2024-09-19 15:02:13.533 | INFO     | __main__:<module>:8 - question 268 obtain answer
2024-09-19 15:02:13.535 | INFO     | __main__:<module>:8 - question 269 obtain answer
2024-09-19 15:02:13.536 | INFO     | __main__:<module>:8 - question 270 obtain answer
2024-09-19 15:02:13.539 | INFO     | __main__:<module>:8 - question 271 obtain answer
2024-09-19 15:02:13.541 | INFO     | __main__:<module>:8 - question 272 obtain answer
2024-09-19 15:02:13.543 | INFO     | __main__:<module>:8 - question 273 obtain answer
2024-09-19 15:02:13.544 | INFO     | __main__:<module>

In [41]:
answers = [a if a != 'None' else str(np.inf) for a in answers]

In [42]:
def mark(pred, real, error: float) -> List[bool]:    
    correct = []
    for p, r in zip(pred, real):
        if p == 'Error':
            continue
        if float(r) != 0:
            if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < error):
                correct.append(True)
            else:
                correct.append(False)
        else:
            if float(p) < error:
                correct.append(True)
            else:
                correct.append(False)
    return correct

In [43]:
print(f"Accuracy under error {1e-2}: {sum(mark(pred_answers, answers, 1e-2)) / len(answers) * 100}")
print(f"Accuracy under error {1e-4}: {sum(mark(pred_answers, answers, 1e-4)) / len(answers) * 100}")
print(f"Accuracy under error {1e-6}: {sum(mark(pred_answers, answers, 1e-6)) / len(answers) * 100}")

Accuracy under error 0.01: 61.458333333333336
Accuracy under error 0.0001: 60.06944444444444
Accuracy under error 1e-06: 60.06944444444444
